In [24]:
import numpy as np

## Activation Function

In [27]:
def sigmoid(x):
    return 1/(1+np.exp(-x))

def tanh(x):
    return np.tanh(x)

## LSTM Cell

In [30]:
class LSTMCell:
    def __init__(self, input_size, hidden_size):
        self.input_size = input_size
        self.hidden_size = hidden_size

        concat_size = input_size + hidden_size

        self.Wf = np.random.randn(hidden_size, concat_size) * 0.1
        self.Wi = np.random.randn(hidden_size, concat_size) * 0.1
        self.Wg = np.random.randn(hidden_size, concat_size) * 0.1
        self.Wo = np.random.randn(hidden_size, concat_size) * 0.1

        self.bf = np.zeros((hidden_size, 1))
        self.bi = np.zeros((hidden_size, 1))
        self.bg = np.zeros((hidden_size, 1))
        self.bo = np.zeros((hidden_size, 1))

    def forward(self, x_t, h_prev, c_prev):
        z = np.vstack((h_prev, x_t))

        f_t = sigmoid(self.Wf @ z + self.bf)
        i_t = sigmoid(self.Wi @ z + self.bi)
        g_t = tanh(self.Wg @ z + self.bg)

        c_t = f_t * c_prev + i_t * g_t

        o_t = sigmoid(self.Wo @ z + self.bo)
        h_t = o_t * tanh(c_t)

        
        return h_t, c_t


In [32]:
class Linear:
    """
    Simple linear layer:
    y = W h + b
    """

    def __init__(self, input_size):
        self.W = np.random.randn(1, input_size) * 0.1
        self.b = np.zeros((1, 1))

    def forward(self, h):
        return self.W @ h + self.b

## Loss Function

In [35]:
def mse_loss(y_pred, y_true):
    return 0.5 * (y_pred - y_true) ** 2


In [37]:
def create_sequences(data, seq_length):
    
    X = []
    y = []

    for i in range(len(data) - seq_length):
        X.append(data[i:i + seq_length])
        y.append(data[i + seq_length])

    return np.array(X), np.array(y)


In [39]:
def forward_sequence(lstm, linear, x_seq):
    
    # Initialize hidden and cell states to zero
    h = np.zeros((lstm.hidden_size, 1))
    c = np.zeros((lstm.hidden_size, 1))

    # Unroll LSTM through time
    for t in range(len(x_seq)):
        x_t = np.array([[x_seq[t]]])  # shape (1, 1)
        h, c = lstm.forward(x_t, h, c)

    # Final prediction
    y_pred = linear.forward(h)

    return y_pred


In [41]:
data = np.arange(0.0, 10.0, 0.1)
seq_length = 4

X, y = create_sequences(data, seq_length)

# Initialize model
input_size = 1
hidden_size = 16

lstm = LSTMCell(input_size, hidden_size)
linear = Linear(hidden_size)

# Forward pass only
for i in range(5):
    x_seq = X[i]
    target = y[i]

    y_pred = forward_sequence(lstm, linear, x_seq)
    loss = mse_loss(y_pred, target)

    print(f"Input: {x_seq} → Target: {target:.2f}, Prediction: {y_pred.item():.4f}")


Input: [0.  0.1 0.2 0.3] → Target: 0.40, Prediction: 0.0013
Input: [0.1 0.2 0.3 0.4] → Target: 0.50, Prediction: 0.0021
Input: [0.2 0.3 0.4 0.5] → Target: 0.60, Prediction: 0.0028
Input: [0.3 0.4 0.5 0.6] → Target: 0.70, Prediction: 0.0035
Input: [0.4 0.5 0.6 0.7] → Target: 0.80, Prediction: 0.0042
